##### *Copyright 2020 Google LLC*
*Licensed under the Apache License, Version 2.0 (the "License")*

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Retrain a classification model for Edge TPU with quant-aware training (TF 1.15)

This notebook uses a set of TensorFlow training scripts to perform transfer-learning on a quantization-aware classification model and then convert it for compatibility with the [Edge TPU](https://coral.ai/products/).

Specifically, this tutorial shows you how to perform [fine-tuning](https://github.com/tensorflow/models/blob/master/research/slim/README.md#fine-tuning-a-model-from-an-existing-checkpoint) on the MobileNet V1 model so it can recognize a new set of classes (five types of flowers), using TensorFlow r1.15.

Beware that, compared to a desktop computer, this training can take much longer in Colab because Colab provides limited resources for long-running operations. So you'll likely see faster training speeds if you [connect this notebook to a local runtime](https://research.google.com/colaboratory/local-runtimes.html), or instead follow the [tutorial to run this training in Docker](https://coral.ai/docs/edgetpu/retrain-classification/) (which includes more documentation about this process).

**Note:** As an alternative approach, check out this other Colab notebook that [retrains a classification model for the Edge TPU using post-training quantization](https://colab.sandbox.google.com/github/google-coral/tutorials/blob/master/retrain_classification_ptq_tf1.ipynb#scrollTo=hRTa3Ee15WsJ) (instead of quantization-aware training).

## Import TensorFlow

In [ ]:
! pip uninstall tensorflow -y
! pip install tensorflow-gpu==1.15

In [ ]:
import tensorflow as tf
print(tf.__version__)

## Clone the model and training repos

In [ ]:
! git clone https://github.com/tensorflow/models.git

In [ ]:
! cd models && git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc

In [ ]:
! git clone https://github.com/google-coral/tutorials.git

! cp tutorials/docker/classification/scripts/* models/research/slim/

## Convert training data to TFRecord

In [ ]:
%cd models/research/slim

In [ ]:
! ./prepare_checkpoint_and_dataset.sh --network_type mobilenet_v1

## Perform transfer-learning

The following script takes a couple hours to finish in Colab. (You can shorten by reducing the steps, but that reduces the final accuracy.)

If you didn't already select "Run all" then you should run all remaining cells now. That will ensure the rest of the notebook completes while you are away, avoiding the chance that the Colab runtime times-out and you lose the training data before you download the model.

In [ ]:
%env NUM_TRAINING_STEPS=300

In [ ]:
! ./start_training.sh --network_type mobilenet_v1 --train_steps $NUM_TRAINING_STEPS

As training progresses, you can see new checkpoint files appear in the `models/research/slim/transfer_learn/train/` directory.

In [ ]:
! ./run_evaluation.sh --network_type mobilenet_v1

## Compile for the Edge TPU

In [ ]:
! ./convert_checkpoint_to_edgetpu_tflite.sh --network_type mobilenet_v1 --checkpoint_num $NUM_TRAINING_STEPS

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

In [ ]:
%cd transfer_learn/models/

! ls

In [ ]:
! edgetpu_compiler output_tflite_graph.tflite

Download the files:

In [ ]:
from google.colab import files

files.download('output_tflite_graph_edgetpu.tflite')
files.download('labels.txt')

If you get a "Failed to fetch" error here, it's probably because the files weren't done saving. So just wait a moment and try again.

Also look out for a browser popup that might need approval to download the files.

## Run the model on the Edge TPU




You can now run the model on your Coral device with acceleration on the Edge TPU.

First, you need some flower photos to try. Remember that you've trained this model to recognize just five flower classes: daisy,
dandelion, roses, sunflowers, and tulips. Here are a couple images you can try (provided by the
[Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html)):

```
wget https://c4.staticflickr.com/3/2856/13169252123_e4c5086ea3_z.jpg -O flower.jpg && \
wget https://c6.staticflickr.com/4/3372/3416475881_726f0d33fe_z.jpg -O flower2.jpg
```

Then, try running an inference using [this example code for the PyCoral API](https://github.com/google-coral/pycoral/blob/master/examples/classify_image.py). Just clone that repo and run the script using the model files you downloaded above (also be sure you have [installed the PyCoral API](https://coral.ai/software/#pycoral-api)):

```
git clone https://github.com/google-coral/pycoral

cd pycoral/examples/

python3 classify_image.py \
  --model output_tflite_graph_edgetpu.tflite  \
  --labels labels.txt \
  --input flower.jpg
```

Check out more examples for running inference at [coral.ai/examples](https://coral.ai/examples/#code-examples/).

## Implementation details



All the scripts used in this notebook come from the following locations:<br>
+  https://github.com/google-coral/tutorials/tree/master/docker/classification/scripts
+  https://github.com/tensorflow/models/tree/r1.13.0/research/slim

More explanation of the steps in this tutorial is available at https://coral.ai/docs/edgetpu/retrain-classification/.